<a href="https://colab.research.google.com/github/JayShekhavat/Fake-news-prediction-ML-Project/blob/main/Fack_news_prediction_project_AI_(NLP)_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Import all required libraries and algorithms**

In [167]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


##**Load dataset by read_csv method**

In [168]:
df = pd.read_csv("/content/FakeNewsNet.csv")

In [169]:
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [170]:
df.sample(5)

,title,news_url,source_domain,tweet_num,real
13114,Saturday Savings: Kourtney Kardashian's Off-th...,https://en.paperblog.com/saturday-savings-kour...,en.paperblog.com,111,1
19286,Taylor Kinney has a new girlfriend after Lady ...,pagesix.com/2016/10/25/taylor-kinney-has-a-new...,pagesix.com,39,0
12943,Murder of Travis Alexander,https://en.wikipedia.org/wiki/Murder_of_Travis...,en.wikipedia.org,80,1
16650,Billy Crystal tore a rib muscle from sneezing,https://pagesix.com/2017/07/18/bill-crystal-to...,pagesix.com,66,1
16503,Younes Bendjima ‘Pities’ Scott Disick — He’s N...,www.pinterest.co.uk/pin/643100021763704904/,www.pinterest.co.uk,11,0


##**Drop some columns which are not important.**

In [171]:
df.drop(columns = ['news_url', 'source_domain', 'tweet_num'], inplace = True, axis = 1)

In [172]:
df.head()

,title,real
0,Kandi Burruss Explodes Over Rape Accusation on...,1
1,People's Choice Awards 2018: The best red carp...,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,1
3,Colombian singer Maluma sparks rumours of inap...,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,1


##**Count the values of dataset**

In [173]:
df.value_counts().sum()

23196

##**Show the shape of dataset**

In [174]:
df.shape

(23196, 2)

###There are 23196 rows and 2 columns in the dataset

##**Check null values in the dataset**

In [175]:
df.isnull().sum()

,0
title,0
real,0


##**Check duplicates in the dataset**

In [176]:
df.duplicated().sum()

1349

###There are 1349 duplicate values

##**Drop the duplicate values in the dataset**

In [177]:
df.drop_duplicates(inplace = True)

In [178]:
df.duplicated().sum()

0

In [179]:
df.sample(4)

,title,real
10703,Here Is The Truth Weather Jennifer And Brad Bu...,0
7421,Jennifer Lawrence Lonely And Partying Out Of C...,0
12140,"Jill Duggar Might Have A Nose Ring, & Fans Are...",1
7146,"Yup, You Can Buy Clothes Worn by Dorinda Medle...",1


In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21847 entries, 0 to 23195
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   21847 non-null  object
 1   real    21847 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 512.0+ KB


##**Check shape of dataset after drop duplicates.**

In [181]:
df.shape

(21847, 2)

###There are 21847 rows and 2 columns remove duplicates

##**Store input feature and target column to text and label respectively**

In [182]:
text = df['title']
label = df['real']

In [183]:
text

,title
0,Kandi Burruss Explodes Over Rape Accusation on...
1,People's Choice Awards 2018: The best red carp...
2,Sophia Bush Sends Sweet Birthday Message to 'O...
3,Colombian singer Maluma sparks rumours of inap...
4,Gossip Girl 10 Years Later: How Upper East Sid...
...,...
23190,See 'Love Actually' Sequel Clip for Red Nose D...
23191,Pippa Middleton wedding: In case you missed it...
23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...
23193,Jessica Chastain Recalls the Moment Her Mother...


In [184]:
print(type(text))

<class 'pandas.core.series.Series'>


In [185]:
print(type(df['title']))

<class 'pandas.core.series.Series'>


In [186]:
df['title'].info()

<class 'pandas.core.series.Series'>
Index: 21847 entries, 0 to 23195
Series name: title
Non-Null Count  Dtype 
--------------  ----- 
21847 non-null  object
dtypes: object(1)
memory usage: 341.4+ KB


In [187]:
label.info()

<class 'pandas.core.series.Series'>
Index: 21847 entries, 0 to 23195
Series name: real
Non-Null Count  Dtype
--------------  -----
21847 non-null  int64
dtypes: int64(1)
memory usage: 341.4 KB


In [188]:
label

,real
0,1
1,1
2,1
3,1
4,1
...,...
23190,1
23191,1
23192,0
23193,1


In [189]:
text = text.tolist()

In [190]:
text

["Kandi Burruss Explodes Over Rape Accusation on 'Real Housewives of Atlanta' Reunion (Video)",
 "People's Choice Awards 2018: The best red carpet looks",
 "Sophia Bush Sends Sweet Birthday Message to 'One Tree Hill' Co-Star Hilarie Burton: 'Breyton 4eva'",
 'Colombian singer Maluma sparks rumours of inappropriate relationship with AUNT',
 'Gossip Girl 10 Years Later: How Upper East Siders Shocked the World and Changed Pop Culture Forever',
 'Gwen Stefani Got Dumped by Blake Shelton Over "Jealousy and Drama" (EXCLUSIVE)',
 'Broward County Sheriff Fired For Lying About Parkland',
 "Amber Rose Shuts Down French Montana Dating Rumors, Calls Rapper Her 'Bruvaaa'",
 "Mindy Kaling makes first post-baby appearance at Disneyland with her 'Wrinkle in Time' co-stars",
 'Katharine McPhee Butchers Tony Nominations: “I Have Not Been Drinking”',
 '‘WAGS Miami’ Stars Ashley Nicole Roberts and Philip Wheeler Are Married',
 'Mel Gibson: Hollywood Pedophiles Have Nowhere Left To Hide',
 'Medium Tyler He

##**Check data types and length of text and label variable**

In [191]:
print(len(text))

21847


In [192]:
print(type(text))

<class 'list'>


In [193]:
print(type(label))

<class 'pandas.core.series.Series'>


##**Sentences convert into lowercase**

In [194]:
lower_stage_1 = []
def lower_case(text):
  for sent in text:
    lower_stage_1.append(str.lower(sent))

In [195]:
lower_case(text)

In [196]:
print(lower_stage_1)

["kandi burruss explodes over rape accusation on 'real housewives of atlanta' reunion (video)", "people's choice awards 2018: the best red carpet looks", "sophia bush sends sweet birthday message to 'one tree hill' co-star hilarie burton: 'breyton 4eva'", 'colombian singer maluma sparks rumours of inappropriate relationship with aunt', 'gossip girl 10 years later: how upper east siders shocked the world and changed pop culture forever', 'gwen stefani got dumped by blake shelton over "jealousy and drama" (exclusive)', 'broward county sheriff fired for lying about parkland', "amber rose shuts down french montana dating rumors, calls rapper her 'bruvaaa'", "mindy kaling makes first post-baby appearance at disneyland with her 'wrinkle in time' co-stars", 'katharine mcphee butchers tony nominations: “i have not been drinking”', '‘wags miami’ stars ashley nicole roberts and philip wheeler are married', 'mel gibson: hollywood pedophiles have nowhere left to hide', 'medium tyler henry addresse

In [197]:
lower_stage_1

["kandi burruss explodes over rape accusation on 'real housewives of atlanta' reunion (video)",
 "people's choice awards 2018: the best red carpet looks",
 "sophia bush sends sweet birthday message to 'one tree hill' co-star hilarie burton: 'breyton 4eva'",
 'colombian singer maluma sparks rumours of inappropriate relationship with aunt',
 'gossip girl 10 years later: how upper east siders shocked the world and changed pop culture forever',
 'gwen stefani got dumped by blake shelton over "jealousy and drama" (exclusive)',
 'broward county sheriff fired for lying about parkland',
 "amber rose shuts down french montana dating rumors, calls rapper her 'bruvaaa'",
 "mindy kaling makes first post-baby appearance at disneyland with her 'wrinkle in time' co-stars",
 'katharine mcphee butchers tony nominations: “i have not been drinking”',
 '‘wags miami’ stars ashley nicole roberts and philip wheeler are married',
 'mel gibson: hollywood pedophiles have nowhere left to hide',
 'medium tyler he

In [198]:
!pip install nltk

In [199]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##**Tokenize the sentences of dataset**

In [200]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [201]:
stage_tokenize_2 = []
for sent in lower_stage_1:
  sent = word_tokenize(sent)
  stage_tokenize_2.append(sent)


In [202]:
print(len(stage_tokenize_2))

21847


In [203]:
stage_tokenize_2

[['kandi',
  'burruss',
  'explodes',
  'over',
  'rape',
  'accusation',
  'on',
  "'real",
  'housewives',
  'of',
  'atlanta',
  "'",
  'reunion',
  '(',
  'video',
  ')'],
 ['people',
  "'s",
  'choice',
  'awards',
  '2018',
  ':',
  'the',
  'best',
  'red',
  'carpet',
  'looks'],
 ['sophia',
  'bush',
  'sends',
  'sweet',
  'birthday',
  'message',
  'to',
  "'one",
  'tree',
  'hill',
  "'",
  'co-star',
  'hilarie',
  'burton',
  ':',
  "'breyton",
  '4eva',
  "'"],
 ['colombian',
  'singer',
  'maluma',
  'sparks',
  'rumours',
  'of',
  'inappropriate',
  'relationship',
  'with',
  'aunt'],
 ['gossip',
  'girl',
  '10',
  'years',
  'later',
  ':',
  'how',
  'upper',
  'east',
  'siders',
  'shocked',
  'the',
  'world',
  'and',
  'changed',
  'pop',
  'culture',
  'forever'],
 ['gwen',
  'stefani',
  'got',
  'dumped',
  'by',
  'blake',
  'shelton',
  'over',
  '``',
  'jealousy',
  'and',
  'drama',
  "''",
  '(',
  'exclusive',
  ')'],
 ['broward',
  'county',
  'sh

##**Remove special charecters**

In [204]:
import re
stage_3_rem_sp_char = []
for sent in stage_tokenize_2:
  cleaned = []
  for word in sent:
    word = re.sub(r'[^\w\s]', '', word)
    if word != '':
       cleaned.append(word)
  stage_3_rem_sp_char.append(cleaned)


In [205]:
print(len(stage_3_rem_sp_char))

21847


In [206]:
print(type(stage_3_rem_sp_char))

<class 'list'>


In [207]:
stage_3_rem_sp_char

[['kandi',
  'burruss',
  'explodes',
  'over',
  'rape',
  'accusation',
  'on',
  'real',
  'housewives',
  'of',
  'atlanta',
  'reunion',
  'video'],
 ['people',
  's',
  'choice',
  'awards',
  '2018',
  'the',
  'best',
  'red',
  'carpet',
  'looks'],
 ['sophia',
  'bush',
  'sends',
  'sweet',
  'birthday',
  'message',
  'to',
  'one',
  'tree',
  'hill',
  'costar',
  'hilarie',
  'burton',
  'breyton',
  '4eva'],
 ['colombian',
  'singer',
  'maluma',
  'sparks',
  'rumours',
  'of',
  'inappropriate',
  'relationship',
  'with',
  'aunt'],
 ['gossip',
  'girl',
  '10',
  'years',
  'later',
  'how',
  'upper',
  'east',
  'siders',
  'shocked',
  'the',
  'world',
  'and',
  'changed',
  'pop',
  'culture',
  'forever'],
 ['gwen',
  'stefani',
  'got',
  'dumped',
  'by',
  'blake',
  'shelton',
  'over',
  'jealousy',
  'and',
  'drama',
  'exclusive'],
 ['broward',
  'county',
  'sheriff',
  'fired',
  'for',
  'lying',
  'about',
  'parkland'],
 ['amber',
  'rose',
  'sh

##**Remove stopword**s

In [208]:
stage_4_rem_stpw = []
for words in stage_3_rem_sp_char:
  cleaned = []
  for word in words:
    if word not in stopwords.words('english'):
      cleaned.append(word)
  stage_4_rem_stpw.append(cleaned)

In [209]:
print(len(stage_4_rem_stpw))

21847


In [210]:
print(type(stage_4_rem_stpw))

<class 'list'>


In [211]:
stage_4_rem_stpw

[['kandi',
  'burruss',
  'explodes',
  'rape',
  'accusation',
  'real',
  'housewives',
  'atlanta',
  'reunion',
  'video'],
 ['people', 'choice', 'awards', '2018', 'best', 'red', 'carpet', 'looks'],
 ['sophia',
  'bush',
  'sends',
  'sweet',
  'birthday',
  'message',
  'one',
  'tree',
  'hill',
  'costar',
  'hilarie',
  'burton',
  'breyton',
  '4eva'],
 ['colombian',
  'singer',
  'maluma',
  'sparks',
  'rumours',
  'inappropriate',
  'relationship',
  'aunt'],
 ['gossip',
  'girl',
  '10',
  'years',
  'later',
  'upper',
  'east',
  'siders',
  'shocked',
  'world',
  'changed',
  'pop',
  'culture',
  'forever'],
 ['gwen',
  'stefani',
  'got',
  'dumped',
  'blake',
  'shelton',
  'jealousy',
  'drama',
  'exclusive'],
 ['broward', 'county', 'sheriff', 'fired', 'lying', 'parkland'],
 ['amber',
  'rose',
  'shuts',
  'french',
  'montana',
  'dating',
  'rumors',
  'calls',
  'rapper',
  'bruvaaa'],
 ['mindy',
  'kaling',
  'makes',
  'first',
  'postbaby',
  'appearance',

##**Stemming the words of sentenves of dataet**

In [212]:
stemming = PorterStemmer()

In [213]:
stage_5_stemming = []
for words in stage_4_rem_stpw:
  words_lst = []
  for word in words:
    words_lst.append(stemming.stem(word))
  stage_5_stemming.append(words_lst)


In [214]:
print(len(stage_5_stemming))

21847


In [215]:
stage_5_stemming

[['kandi',
  'burruss',
  'explod',
  'rape',
  'accus',
  'real',
  'housew',
  'atlanta',
  'reunion',
  'video'],
 ['peopl', 'choic', 'award', '2018', 'best', 'red', 'carpet', 'look'],
 ['sophia',
  'bush',
  'send',
  'sweet',
  'birthday',
  'messag',
  'one',
  'tree',
  'hill',
  'costar',
  'hilari',
  'burton',
  'breyton',
  '4eva'],
 ['colombian',
  'singer',
  'maluma',
  'spark',
  'rumour',
  'inappropri',
  'relationship',
  'aunt'],
 ['gossip',
  'girl',
  '10',
  'year',
  'later',
  'upper',
  'east',
  'sider',
  'shock',
  'world',
  'chang',
  'pop',
  'cultur',
  'forev'],
 ['gwen',
  'stefani',
  'got',
  'dump',
  'blake',
  'shelton',
  'jealousi',
  'drama',
  'exclus'],
 ['broward', 'counti', 'sheriff', 'fire', 'lie', 'parkland'],
 ['amber',
  'rose',
  'shut',
  'french',
  'montana',
  'date',
  'rumor',
  'call',
  'rapper',
  'bruvaaa'],
 ['mindi',
  'kale',
  'make',
  'first',
  'postbabi',
  'appear',
  'disneyland',
  'wrinkl',
  'time',
  'costar'],


##**Lemmatized the words of dataset**

In [216]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [217]:

from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()


In [218]:
lemmetized_text = []
for sent in stage_5_stemming:
  w = []
  for word in sent:
    w.append(lemmatizer.lemmatize(word, pos = 'a'))
  lemmetized_text.append(" ".join(w))

In [219]:

df['text'] = lemmetized_text

In [220]:
label

,real
0,1
1,1
2,1
3,1
4,1
...,...
23190,1
23191,1
23192,0
23193,1


In [221]:
df['real'] = label

In [222]:
df.head()

,title,real,text
0,Kandi Burruss Explodes Over Rape Accusation on...,1,kandi burruss explod rape accus real housew at...
1,People's Choice Awards 2018: The best red carp...,1,peopl choic award 2018 best red carpet look
2,Sophia Bush Sends Sweet Birthday Message to 'O...,1,sophia bush send sweet birthday messag one tre...
3,Colombian singer Maluma sparks rumours of inap...,1,colombian singer maluma spark rumour inappropr...
4,Gossip Girl 10 Years Later: How Upper East Sid...,1,gossip girl 10 year late upper east side shock...


In [223]:
x = df['title']
y = df['real']

##**Split dataset into train test**

In [224]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [225]:
print(type(x_train))

<class 'pandas.core.series.Series'>


In [226]:
print(type(x_train))

<class 'pandas.core.series.Series'>


##**Vectorize the dataset**

In [227]:
x_train

,title
21527,Kardashian blonde moments we loved
15752,transcript accessed via Nexis Mitt Romney Send...
4070,‘Will & Grace’ could extend its Golden Globes ...
20262,14 ways the Marvel Cinematic Universe could ch...
21127,Tyrese Gibson posts video of The Rock dissing ...
...,...
12442,Does James Kennedy Still Have Feelings for Lal...
22902,Meghan Markle Channels Princess Diana With a T...
5501,Beyonce & Kim Kardashian Met Up at Serena Will...
868,They Didn't Mean It Like That: Actors Defendin...


In [228]:
x_test

,title
4570,YouTube star Austin Jones charged with child p...
19262,Marvel Studios: When Does Each Actor's Contrac...
19121,5 Reasons to Boycott Khloe Kardashian’s New Sh...
10791,2017 American Music Awards: What You Didn’t Se...
3256,Emmy Award Winner Louis J. Horvitz To Direct E...
...,...
14356,"Disney Fans, Brace Yourselves: Frozen's Broadw..."
10290,A Timeline of Pippa Middleton and James Matthe...
10461,Gigi Hadid Drops Out of the 2017 Victoria’s Se...
249,05/27/99 Committee on the Judiciary


In [229]:
y_train

,real
21527,1
15752,1
4070,1
20262,1
21127,1
...,...
12442,1
22902,1
5501,1
868,1


In [230]:
y_test

,real
4570,1
19262,1
19121,1
10791,1
3256,1
...,...
14356,1
10290,1
10461,1
249,1


In [231]:

vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [232]:
x_train

<16385x15623 sparse matrix of type '<class 'numpy.float64'>'
	with 177526 stored elements in Compressed Sparse Row format>

In [233]:
x_test

<5462x15623 sparse matrix of type '<class 'numpy.float64'>'
	with 57152 stored elements in Compressed Sparse Row format>

In [234]:
print(type(x_train))

<class 'scipy.sparse._csr.csr_matrix'>


##**Apply Logistic Regression algorithm**

In [235]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
model = log_reg.fit(x_train, y_train)

In [236]:
y_predict =model.predict(x_test)

In [237]:
lr_accuracy = accuracy_score(y_predict, y_test)

In [238]:
print("Logistic Regression algorithm Accuracy : ", lr_accuracy)

Logistic Regression algorithm Accuracy :  0.827718784328085


###Logistic Regression algorithm give **0.8278** accuracy

##**Apply Naive Bayes algorithm**

In [239]:
from sklearn.naive_bayes import MultinomialNB
bayes_algo = MultinomialNB()
bayes_algo_model = bayes_algo.fit(x_train, y_train)

In [240]:
y_predict =bayes_algo_model.predict(x_test)

In [241]:
nb_accuracy = accuracy_score(y_predict, y_test)

In [242]:
print("Naive bayes algorithm Accuracy : ", nb_accuracy)

Naive bayes algorithm Accuracy :  0.7993409007689491


###Naive Bayes algorithm give **0.7994** accuracy

##**Apply Support Vector Machine algorithm**

In [243]:
from sklearn.svm import SVC
svc_algo = SVC(kernel='linear', C=1.0, random_state=42)
model = svc_algo.fit(x_train, y_train)

In [244]:
y_predict =model.predict(x_test)

In [245]:
svm_accuracy = accuracy_score(y_predict, y_test)

In [246]:
print("Support Vector Machine algorithm Accuracy : ", svm_accuracy)

Support Vector Machine algorithm Accuracy :  0.8359575247162212


###Apply Support Vector Machine algorithm give **0.8350** accuracy

##**Apply k-Nearest Neighbors (k-NN) algorithm**

In [256]:
from sklearn.neighbors import KNeighborsClassifier
knn_algo = KNeighborsClassifier(n_neighbors=5)
model = knn_algo.fit(x_train, y_train)

In [257]:
y_predict =model.predict(x_test)

In [258]:
knn_accuracy = accuracy_score(y_predict, y_test)

In [259]:
print("k-Nearest Neighbors (k-NN) algorithm Accuracy : ", knn_accuracy)

k-Nearest Neighbors (k-NN) algorithm Accuracy :  0.8084950567557672


###k-Nearest Neighbors (k-NN) algorithm give **0.8084..** accuracy at k=5

#**Conclusion**
I made a project on NLP, Fack news prediction project. I used 4 algorithms in this project but Support Vector Machine algorithm give high accuracy which is (**0.8350**) so this algorithm is best for this project.